## Модуль А.  Парсинг и предобработка данных

### Введение

нана

#### Импортирование библиотек

In [1]:
from bs4 import BeautifulSoup as bs
import requests 
import pandas as pd
import io
import glob
import re
import nltk
import matplotlib.pyplot as plt
import pymorphy2
import fitz
import string
import re
from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from nltk.tokenize import word_tokenize
from tqdm.auto import tqdm, trange

#### Установка библиотек

In [2]:
pip install pymystem3

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install fitz

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install PyMuPDF

Note: you may need to restart the kernel to use updated packages.


### 1.1 Парсинг данных

In [5]:
# создание списка для дальнейшей записи в него данных и преобразования в DataFrame
df = {"TitleCompany" : [],"DescriptionCompany" : [], "ReitingCompany" : [],"CategoriesCompany" : [],"TitleTextPosts" : []}

In [6]:
# выводим общую информацию о пдф файле
pdf_document = r"C:\Users\essww\OneDrive\Рабочий стол\Компании.pdf"
doc = fitz.open(pdf_document)
print("Исходный документ: ", doc)
print("\nКоличество страниц: %i\n\n------------------\n\n" % doc.page_count)
print(doc.metadata)
text_pages = ""
for current_page in range(len(doc)):
    page = doc.load_page(current_page)
    page_text = page.get_text("text")
    # записываем текст пдф файла
    text_pages += page.get_text("text")
    print("Стр. ", current_page+1, "\n")
    print(page_text)# выводим общую информацию о пдф файле

Исходный документ:  Document('C:\Users\essww\OneDrive\Рабочий стол\Компании.pdf')

Количество страниц: 6

------------------


{'format': 'PDF 1.7', 'title': 'Aspose', 'author': 'Aspose', 'subject': 'Aspose', 'keywords': '', 'creator': 'Aspose Pty Ltd.', 'producer': 'Aspose.PDF for .NET 23.2.0', 'creationDate': "D:20230322184058+00'00'", 'modDate': "D:20230322184137+00'00'", 'trapped': '', 'encryption': None}
Стр.  1 

RUVDS.com
VDS/VPS-хостинг. Скидка 15% по коду HABR15
Пишет в хабы:  Читальный зал,�DIY или Сделай сам,�Научно-популярное,�
Электроника для начинающих,�Системное администрирование
2393.91
72K
Selectel
IT-инфраструктура для бизнеса
Пишет в хабы:  Компьютерное железо,�Гаджеты,�IT-инфраструктура,�
IT-компании,�Производство и разработка электроники
1935.27
40K
Timeweb Cloud
Облачная платформа для разработчиков и бизнеса
Пишет в хабы:  Читальный зал,�Научно-популярное,�JavaScript,�
DIY или Сделай сам,�История IT
1542.55
1.1K
OTUS
Цифровые навыки от ведущих экспертов
Пишет в ха

In [7]:
# Разделяем текст на строки
text_pages = text_pages.split("\n")

In [8]:
text_pages

['RUVDS.com',
 'VDS/VPS-хостинг. Скидка 15% по коду HABR15',
 'Пишет в хабы:  Читальный зал,�DIY или Сделай сам,�Научно-популярное,�',
 'Электроника для начинающих,�Системное администрирование',
 '2393.91',
 '72K',
 'Selectel',
 'IT-инфраструктура для бизнеса',
 'Пишет в хабы:  Компьютерное железо,�Гаджеты,�IT-инфраструктура,�',
 'IT-компании,�Производство и разработка электроники',
 '1935.27',
 '40K',
 'Timeweb Cloud',
 'Облачная платформа для разработчиков и бизнеса',
 'Пишет в хабы:  Читальный зал,�Научно-популярное,�JavaScript,�',
 'DIY или Сделай сам,�История IT',
 '1542.55',
 '1.1K',
 'OTUS',
 'Цифровые навыки от ведущих экспертов',
 'Пишет в хабы:  Программирование,�Java,�Машинное обучение,�',
 'Тестирование веб-сервисов,�Python',
 '832.56',
 '103K',
 'Яндекс',
 'Как мы делаем Яндекс',
 'Добавить компанию',
 'Все потоки',
 'СТАТЬИ',
 '+64',
 'НОВОСТИ',
 '+50',
 'ХАБЫ',
 'АВТОРЫ',
 'КОМПАНИИ',
 'Название ',
 'Рейтинг ',
 'Подписчики',
 'КАК СТАТЬ АВТОРОМ',
 'Пишет в хабы:  Машинн

In [9]:
# Выполняем проверку на 6 индексе, т.к. на каждом 6 индексе находится нужное название компании
for i in range(0,len(text_pages),6):
    print(text_pages[i])
for i in range(len(text_pages)):
    print(i, text_pages[i])

RUVDS.com
Selectel
Timeweb Cloud
OTUS
Яндекс
НОВОСТИ
Рейтинг 
164K
9.6K
54K
167K
953
353
24K
58K
519
24K
76K
28K
40K
14K
41K
47K
Ваш
Все
Hardware
Mass media
E-commerce
Producing multimedia content
Search Engines
Games and entertainment
Публикации
Устройство сайта
Корпоративный блог
Мегапроекты
0 RUVDS.com
1 VDS/VPS-хостинг. Скидка 15% по коду HABR15
2 Пишет в хабы:  Читальный зал,�DIY или Сделай сам,�Научно-популярное,�
3 Электроника для начинающих,�Системное администрирование
4 2393.91
5 72K
6 Selectel
7 IT-инфраструктура для бизнеса
8 Пишет в хабы:  Компьютерное железо,�Гаджеты,�IT-инфраструктура,�
9 IT-компании,�Производство и разработка электроники
10 1935.27
11 40K
12 Timeweb Cloud
13 Облачная платформа для разработчиков и бизнеса
14 Пишет в хабы:  Читальный зал,�Научно-популярное,�JavaScript,�
15 DIY или Сделай сам,�История IT
16 1542.55
17 1.1K
18 OTUS
19 Цифровые навыки от ведущих экспертов
20 Пишет в хабы:  Программирование,�Java,�Машинное обучение,�
21 Тестирование веб-сервис

In [10]:
# создаем DataFrame
df = pd.DataFrame({"TitleCompany": [], "Description": [], "Reiting": [], "Categories": []})

In [11]:
TitleCompany = []
Description = []
Reiting = []
Categories = []

In [12]:
# т.к. начиная с 26 по 39 индекса, и с 132 до коннца начинается ненужная информация, мы ее удаляем
text_new_pages = text_pages[0:26] + text_pages[39:132]
for i in tqdm(range(0,len(text_new_pages),6)):
    TitleCompany.append(text_new_pages[i])
    Description.append(text_new_pages[i+1])
    Reiting.append(text_new_pages[i+4])
    Categories.append(text_new_pages[i+3].replace('�', ' '))

  0%|          | 0/20 [00:00<?, ?it/s]

In [13]:
# Запись данных в df
df["TitleCompany"] = TitleCompany
df["Description"] = Description
df["Reiting"] = Reiting
df["Categories"] = Categories

In [14]:
df

,TitleCompany,Description,Reiting,Categories
0,RUVDS.com,VDS/VPS-хостинг. Скидка 15% по коду HABR15,2393.91,"Электроника для начинающих, Системное админист..."
1,Selectel,IT-инфраструктура для бизнеса,1935.27,"IT-компании, Производство и разработка электро..."
2,Timeweb Cloud,Облачная платформа для разработчиков и бизнеса,1542.55,"DIY или Сделай сам, История IT"
3,OTUS,Цифровые навыки от ведущих экспертов,832.56,"Тестирование веб-сервисов, Python"
4,Яндекс,Как мы делаем Яндекс,721.53,"Высокая производительность, Программирование, ..."
5,Open Data Science,Крупнейшее русскоязычное Data Science сообщество,612.13,"Natural Language Processing, Open source, Big ..."
6,ГК ЛАНИТ,Ведущая многопрофильная группа ИТ-компаний в РФ,569.92,"Управление персоналом, Машинное обучение, Прог..."
7,VK,"Технологии, которые объединяют",536.95,Облачные сервисы
8,getmatch,"Рассказываем о том, как строить карьеру в IT",436.91,"IT-эмиграция, Урбанизм"
9,Wiren Board,Оборудование для автоматизации и мониторинга,411.66,"DIY или Сделай сам, Компьютерное железо"


### Мы переходим к парсингу, после того как получили список нужных нам компаний

In [15]:
dfHabr = {"TitleCompany": [], "Description": [], "Reiting": [], "Categories": [], "TextPost": []}

In [16]:
for i in range(len(df)):
    print(i, df['TitleCompany'][i])

0 RUVDS.com
1 Selectel
2 Timeweb Cloud
3 OTUS
4 Яндекс
5 Open Data Science
6 ГК ЛАНИТ
7 VK
8 getmatch
9 Wiren Board
10 FirstVDS
11 Конференции Олега Бунина (Онтико)
12 AGIMA
13 Альфа-Банк
14 Postgres Professional
15 Ozon Tech
16 TINKOFF
17 Хабр Карьера
18 JUG Ru Group
19 Southbridge


In [17]:
pagenum = 1
for i in tqdm(range(25)):
    url = "https://habr.com/ru/all/page" + str(pagenum) + '/'
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    pages = soup.find_all('h2', class_='tm-title tm-title_h2')
    for i in pages:
        if 'companies' in str(i.a.get('href')):
            url = 'https://habr.com' + str(i.a.get('href'))
            page = requests.get(url)
            soup = bs(page.text, 'html.parser')
            TitleCompany = soup.find('a', class_='tm-company-snippet__title')
            for i in range(20):
                if TitleCompany.text in df['TitleCompany'][i]:
                    dfHabr["TitleCompany"].append(df['TitleCompany'][i])
                    dfHabr["Description"].append(df['Description'][i])
                    dfHabr["Reiting"].append(df['Reiting'][i])
                    dfHabr["Categories"].append(df['Categories'][i])
                    TextPost = soup.find('div', class_='tm-article-body')
                    TextPost.text.replace('\n', '')
                    TextPost.text.replace('\r', '')
                    dfHabr["TextPost"].append(TextPost.text)
    pagenum = pagenum + 1

  0%|          | 0/25 [00:00<?, ?it/s]

In [18]:
dfHabr1 = pd.DataFrame(data=dfHabr)

In [19]:
dfHabr1.head(10)

,TitleCompany,Description,Reiting,Categories,TextPost
0,RUVDS.com,VDS/VPS-хостинг. Скидка 15% по коду HABR15,2393.91,"Электроника для начинающих, Системное админист...",\r\nОбщий обзор новейшего алгоритма стеганогр...
1,Timeweb Cloud,Облачная платформа для разработчиков и бизнеса,1542.55,"DIY или Сделай сам, История IT",\r\nЭто было в студенческие годы моих родител...
2,Selectel,IT-инфраструктура для бизнеса,1935.27,"IT-компании, Производство и разработка электро...","\n\r\nИ по-другому это явление не назовешь, и..."
3,RUVDS.com,VDS/VPS-хостинг. Скидка 15% по коду HABR15,2393.91,"Электроника для начинающих, Системное админист...",\r\nЗаконы физики говорят нам обо всех возмож...
4,RUVDS.com,VDS/VPS-хостинг. Скидка 15% по коду HABR15,2393.91,"Электроника для начинающих, Системное админист...","\r\n«Если что-то существует, его уже кто-то н..."
5,Timeweb Cloud,Облачная платформа для разработчиков и бизнеса,1542.55,"DIY или Сделай сам, История IT","\n\r\nС юбилеем, дорогая Элизабет!\n\r\nДобро..."
6,Selectel,IT-инфраструктура для бизнеса,1935.27,"IT-компании, Производство и разработка электро...","\n\r\nКак и в случае мини-ПК, производители о..."
7,Southbridge,Обеспечиваем стабильную работу highload-проектов,272.35,"Карьера в IT-индустрии, Программирование",Автор делится своим опытом установки Kubernet...
8,Альфа-Банк,Лучший мобильный банк по версии Markswebb,311.6,"Машинное обучение, JavaScript, Карьера в IT-ин...",Привет! Меня зовут Татьяна Руфанова. Сегодня ...
9,RUVDS.com,VDS/VPS-хостинг. Скидка 15% по коду HABR15,2393.91,"Электроника для начинающих, Системное админист...",\r\nVue позволяет разработчикам писать более ...


### 1.2 Формирование структуры набора данных

### Описание структуры данных

Изначальные признаки: 

__TitleCompany__ - Название компании

__Description__ - Описание компании

__Reiting__ - Рейтинг компании

__Categories__ - Категории компании

__Date__ - Дата опубликования статьи (в будущем удален, как ненужный признак)

__TextPost__ - необработанный текст (в будущем удален, как ненужный признак)

__TitleCompany__ - Название компании

__Description__ - Описание компании

__Reiting__ - Рейтинг компании

__Categories__ - Категории компании

__TextPostLemat__ - обработанный, лематизированный текст 

__TextPostToken__ - обработанный, токенизированный и лематизированный текст (лематизирован другим способом нежели TextPostLemat)

### 1.3 Предварительная обработка текстовых данных  

In [20]:
#вывод необработанного текста
dfHabr1['TextPost'][0]

' \r\nОбщий обзор новейшего алгоритма стеганографии, позволяющего бесследно скрывать информацию внутри некоего контента. В статье мы разберём, на чём он основывается, для решения каких задач предназначен, какие уязвимости имеет и под какой лицензией будет распространяться.\n\r\nВ современную цифровую эпоху потребность в безопасной коммуникации стала как никогда актуальной. Независимо от того, говорим ли мы про обмен личными или рабочими сообщениями, угроза кибератак и взлома данных вызывает постоянное беспокойство. Криптография, представляющая собой технику шифрования информации с целью её защиты от несанкционированного доступа, уже давно служит в качестве способа обеспечения безопасного обмена сообщениями. Однако этот метод всё же остаётся уязвимым для атак и обнаружения, оставляя чувствительную информацию под угрозой. \n\r\nЗдесь на помощь приходит стеганография. Стеганография – это техника сокрытия чувствительной информации внутри безобидного контента, например изображения или аудио

In [21]:
morph = pymorphy2.MorphAnalyzer()

In [22]:
# метод удаления пунктуации
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

# метод удаления чисел
def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

# метод удаления двойных пробелов
def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)

# метод удаления оставшихся чисел
def remove_notalpha(text):
    return ''.join([i if i.isalpha() else ' ' for i in text])

def remove_english_word(text):
    return ''.join(re.sub(r"[^а-яА-Я\s]+", "",text))

# метод удаления оставшихся символов
st = '❯\xa0'
def remove_othersymbol(text):
    return ''.join([ch if ch not in st else ' ' for ch in text])

# метод токенизирования
def tokenize(text):
    t = word_tokenize(text)
    return [token for token in t if token not in russian_stopwords]

def remove_small_text(text):
    words = text.split()
    return ' '.join([i if len(i) >=3 else ' ' for i in words])

mystem = Mystem() 
# добавление стопслов
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', 'быть', 'r', 'n', 'а', 'мы', 'с', 'для', 'ещё', 'его', 'также', 'к', 'тем', 'кто', 'чтобы', 'но', 'они', 'будут', 'так', 'где', 'один', 'он ', 'и', 'на', 'но', 'или', 'либо', 'это', 'мб', 'далее', 'дв', 'свой', 'ваш','всё', 'очень', 'её', 'ещё', 'вообще', 'наш', 'который'])

In [23]:
# обработка текста с лематизацией для токенизации
prep_text = [tokenize(remove_multiple_spaces(remove_small_text(remove_notalpha(remove_othersymbol(remove_english_word(remove_numbers(remove_punctuation(text.lower())))))))) for text in tqdm(dfHabr1['TextPost']) if text not in russian_stopwords]

  0%|          | 0/91 [00:00<?, ?it/s]

In [24]:
# обработка текста без токенизации и лематизации для отдельной колонки с лематизацией
prep_text1 = [remove_multiple_spaces(remove_small_text(remove_notalpha(remove_othersymbol(remove_english_word(remove_numbers(remove_punctuation(text.lower()))))))) for text in tqdm(dfHabr1['TextPost']) if text not in russian_stopwords]

  0%|          | 0/91 [00:00<?, ?it/s]

In [25]:
dfHabr1['TextPostLemat'] = prep_text1
dfHabr1['TextPostToken'] = prep_text

In [26]:
dfHabr1.drop(columns=['TextPost'], axis= 1 , inplace= True )

### Лемматизация

In [27]:
lemm_texts_list = []
for text in tqdm(dfHabr1['TextPostLemat']):
    try:
        text_lem = mystem.lemmatize(text)
        tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
        text = " ".join(tokens)
        lemm_texts_list.append(text)
    except Exception as e:
        print(e)
        
    
dfHabr1['TextPostLemat'] = lemm_texts_list

  0%|          | 0/91 [00:00<?, ?it/s]

In [ ]:
dfHabr1

In [29]:
# сохраняем DataFrame в файл
dfHabr1.to_csv('Report1.csv', index=False)

### Вывод